In [3]:
!pip install pywedge

  Obtaining dependency information for pywedge from https://files.pythonhosted.org/packages/dc/f4/2d457a2fdb39b48ea78f7de6306a8d45d8c7650c8c8e2cad2dd935bad2d7/pywedge-0.5.1.8-py3-none-any.whl.metadata
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
  Obtaining dependency information for catboost>0.24 from https://files.pythonhosted.org/packages/e8/37/3afd3c02798734efcd7840bfa872d3efc06f5d5c92f9613fea3ff5b4311f/catboost-1.2.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for graphviz from https://files.pythonhosted.org/packages/de/5e/fcbb22c68208d39edff467809d06c9d81d7d27426460ebc598e55130c1aa/graphviz-0.20.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.1 MB 2.4 MB/s eta 0:00:43
   -------------------------

In [5]:
!pip install dash
!pip install pandas
!pip install plotly

  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/d1/41/fdbab78ba4f51015d14168a08fd78804ebcbf9368dd790b49286dd15b3fe/dash-2.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-html-components==2.0.0 from https://files.pythonhosted.org/packages/75/65/1b16b853844ef59b2742a7de74a598f376ac0ab581f0dcc34db294e5c90e/dash_html_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-core-components==2.0.0 from https://files.pythonhosted.org/packages/00/9e/a29f726e84e531a36d56cff187e61d8c96d2cc253c5bcef9a7695acb7e6a/dash_core_components-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for dash-table==5.0.0 from https://files.pythonhosted.org/packages/da/ce/43f77dc8e7bbad02a9f88d07bf794eaf68359df756a28bb9f2f78e255bb1/dash_table-5.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for retrying from https://files.pythonhosted.org/packages/8f/04/9e36f28be4c0532c0e9207ff9dc01fb

In [7]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "SpaceX Launch Dashboard"

# Create the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),
    html.Div([
        html.Label("Select Launch Site:"),
        dcc.Dropdown(id='site-dropdown',
             options=[{'label': site, 'value': site} for site in ['All Sites'] + list(data['Launch Site'].unique())],
             value='All Sites',
             placeholder='Select a Launch Site here',
             searchable=True)
    ]),
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={0: '0', 100: '100'},
        value=[0, 10000]
    ),
    dcc.Graph(id='success-pie-chart'),
    dcc.Graph(id='success-payload-scatter-chart')
])

# Define callback to update the pie chart
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def update_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(data, values='class', names='Launch Site', title='Success Rate by Launch Site')
    else:
        filtered_df = data[data['Launch Site'] == entered_site]
        fig = px.pie(filtered_df, values='class', names='Booster Version', title='Success Rate by Booster Version')
    return fig

# Define callback to update the scatter chart
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def update_scatter_chart(entered_site, payload_range):
    if entered_site == 'ALL':
        fig = px.scatter(data, x='Payload Mass (kg)', y='class', color='Booster Version', title='Payload vs. Outcome for All Sites')
    else:
        filtered_df = data[data['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version', title=f'Payload vs. Outcome for {entered_site}')
    fig.update_xaxes(range=payload_range)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8050, debug=True)
